In [ ]:
#| default_exp prepare_s2a_dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import sys
import os
import itertools
from pathlib import Path

import numpy as np
import torch
import torchaudio
import torch.nn.functional as F
from torch.profiler import profile, record_function, ProfilerActivity

from fastprogress import progress_bar
from fastcore.script import *

import whisper
from whisperspeech import vad, wh_transcribe, vq_stoks, extract_acoustic
import webdataset as wds

# S2A dataset preparation

We take a webdataset shard and extract acoustic and semantic tokens from it.

We don't use the VAD data since the S2A should work on any random 30 second window.

In [ ]:
#| export
def flac_to_s2a_name(input):
    if '-flac-' in input:
        return input.rsplit("/", 1)[1].replace('flac', 's2a') + ".gz"
    else:
        return input.rsplit("/", 1)[1].replace('raw', 's2a') + ".gz"

In [ ]:
flac_to_s2a_name('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar')

'librilight-large-6454-s2a-000000.tar.gz'

In [ ]:
#| exporti
def resampler(newsr = 24000, key = 'samples_24k'):
    _last_sr = None
    tform = None
    
    def _resample(samples):
        for s in samples:
            sr = s['sample_rate']
            if sr != newsr:
                if sr != _last_sr: tform = torchaudio.transforms.Resample(sr, newsr)
                s[key] = tform(s['samples'])
            else:
                s[key] = s['samples']
            yield s
    
    return _resample

In [ ]:
ds = wds.WebDataset(["/data2/libritts-r-raw-000002.tar"]).compose(
        wds.decode(wds.torch_audio),
        wds.select(lambda x: 'wav' in x or 'flac' in x),
        vq_stoks.merge_in(vq_stoks.derived_dataset('/data2/processed/', 'vad')),
        wds.map_dict(**{"vad.npy":wh_transcribe.chunk_merger}),
        lambda x: wh_transcribe.split_to_chunks(x),
)

In [ ]:
for x in progress_bar(ds, total='noinfer'): pass
x

{'__key__': 'train-clean-360/1638/84447/1638_84447_000100_000001_000',
 '__url__': '/data2/libritts-r-raw-000002.tar',
 'i': 0,
 'imax': 0,
 'tstart': 0.00844,
 'tend': 3.232,
 'total_seconds': 3.28,
 'lpad': 0,
 'rpad': 642624,
 'lpad_s': 0.0,
 'rpad_s': 26.776,
 'samples': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'sample_rate': 24000}

In [ ]:
#| exporti
@call_parse
def prepare_s2a(
    input:str,  # FLAC webdataset file path (or - to read the names from stdin)
    proc_dataset_path:Path, # processed VAD files path
    output:str=None, # output file name
    vq_model:str="collabora/spear-tts-pytorch:whisper-vq-stoks.model", # the model path (use repo_id:filename to download it from hugginface)
    n_samples:int=None, # process a limited amount of samples
    batch_size:int=1, # process several segments at once
    fix_dots:bool=False, # fix dots in file names
):
    if ":" in vq_model:
        repo, fname = vq_model.split(":", 1)
        vq_model = vq_stoks.RQBottleneckTransformer.load_model(repo, fname).cuda()
    else:
        vq_model = vq_stoks.RQBottleneckTransformer.load_model(local_filename=vq_model).cuda()
    amodel = extract_acoustic.load_model()
    amodel.set_target_bandwidth(3)

    if input == "-":
        input = [f.strip() for f in sys.stdin.readlines()]
        assert output, "please provide the output shard name"
    else:
        if output is None: output = flac_to_s2a_name(input)
        input = [input]
        
    total = n_samples//batch_size if n_samples else 'noinfer'

    ds = wds.WebDataset(input, shardshuffle=True, rename_files=vad.fix_dots_in_names if fix_dots else None).compose(
        wds.decode(wds.torch_audio),
        wds.select(lambda x: 'wav' in x or 'flac' in x),
        vq_stoks.merge_in(vq_stoks.derived_dataset(proc_dataset_path, 'vad')),
        wds.map_dict(**{"vad.npy":wh_transcribe.chunk_merger}),
        lambda x: wh_transcribe.split_to_chunks(x),
        resampler(),
        resampler(16000, 'samples_16k'),
        wds.to_tuple('__key__', 'rpad_s', 'samples_16k', 'samples_24k'),
        wds.batched(64),
    )

    dl = wds.WebLoader(ds, num_workers=4, batch_size=None).unbatched().shuffle(2000).batched(batch_size)

    speakers = set()
    tmp = output+".tmp"
    with wds.TarWriter(tmp) as sink:
        for keys, rpad_ss, samples, samples24k in progress_bar(dl, total=total):
            with record_function('to_cuda'):
                samples, samples24k = samples.cuda(), samples24k.unsqueeze(1).cuda()
            with record_function('encodec'):
                atoks = amodel.encode(samples24k)[0][0]
            with record_function('vq_stoks'):
                stoks = vq_model.encode_audio(samples)
            with record_function('from_cuda'):
                atoks, stoks = atoks.cpu().numpy().astype(np.int16), stoks.cpu().numpy().astype(np.int16)
            for key, rpad_s, _atoks, _stoks in zip(keys, rpad_ss, atoks, stoks):
                speakers.add(key.split('/')[1])
                sink.write({
                    "__key__": key,
                    "atoks.npy": _atoks[:,:int(-rpad_s * 75)],
                    "stoks.npy": _stoks[:int(-rpad_s * 25)],
                })
    with open(output+".speakers.txt", "w") as f: f.write("\n".join(speakers))
    if not n_samples:
        os.rename(tmp, output)

In [ ]:
prepare_s2a('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar', '/data/whisperspeech-processed-wds/', vq_model='vqmodel-4e-hyptuned-32gpu.model', n_samples=5)

In [ ]:
## Batch size tests

In [ ]:
prepare_s2a('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar', '/data/whisperspeech-processed-wds/', vq_model='vqmodel-4e-hyptuned-32gpu.model', n_samples=1000)

In [ ]:
prepare_s2a('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar', '/data/whisperspeech-processed-wds/', vq_model='vqmodel-4e-hyptuned-32gpu.model', n_samples=1000, batch_size=2)

In [ ]:
prepare_s2a('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar', '/data/whisperspeech-processed-wds/', vq_model='vqmodel-4e-hyptuned-32gpu.model', n_samples=1000, batch_size=4)

In [ ]:
prepare_s2a('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar', '/data/whisperspeech-processed-wds/', vq_model='vqmodel-4e-hyptuned-32gpu.model', n_samples=1000, batch_size=8)

In [ ]:
# stoks only
prepare_s2a('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar', '/data/whisperspeech-processed-wds/', vq_model='vqmodel-4e-hyptuned-32gpu.model', n_samples=1000, batch_size=4)

In [ ]:
# atoks only
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    prepare_s2a('/data/whisperspeech-wds/librilight-large-6454-flac-000000.tar', '/data/whisperspeech-processed-wds/', vq_model='vqmodel-4e-hyptuned-32gpu.model', n_samples=10, batch_size=1)
prof.export_chrome_trace("trace-bs1.json")

STAGE:2023-10-06 14:25:45 71030:71030 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


STAGE:2023-10-06 14:25:47 71030:71030 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-06 14:25:47 71030:71030 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [ ]:
!ls -lh librilight-large-6454-s2a-000000.tar.gz.tmp
!tar -tf librilight-large-6454-s2a-000000.tar.gz.tmp

-rw-r--r-- 1 root root 1.3M Oct  6 09:17 librilight-large-6454-s2a-000000.tar.gz.tmp
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_000.atoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_000.stoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_001.atoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_001.stoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_002.atoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_002.stoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_003.atoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_003.stoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_004.atoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_004.stoks.npy
large/6454/abaft_funnel_1307_librivox_64kb_mp3/funnel_01_kipling_64kb_005.a

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()